In [127]:
# This file is used to test how to get data from the Frax API and supplied info.
# We may also sanity check or use chain access via a local node or blockchain scanner
# if that's necessary

In [2]:
from cache import cache
from collections import defaultdict
from decimal import Decimal
import httpx
import json
from pprint import pprint
import sys
from web3 import Web3

In [31]:
# Get Gauge Pools and sort by gauge_index
req = cache.get('https://api.frax.finance/gauge/info')
gauges = json.loads(req['value'])
'''
# A gauge looks like:

{'address': '0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0',
 'fxs_next': 39981.83490806742,
 'fxs_now': 33694.5538302253,
 'gauge_index': 0,
 'gauge_weight': '17127149135922894237322600',
 'name': 'Uniswap V3 FRAX/USDC',
 'relative_weight_next_pct': 45.69352560921991,
 'relative_weight_next_raw': '456935256092199104',
 'relative_weight_now_pct': 38.508061520257485,
 'relative_weight_now_raw': '385080615202574845'}
'''
gauges.sort(key=lambda g: g['gauge_index'])
for gauge in gauges:
  print(gauge['name'])

Uniswap V3 FRAX/USDC
mStable FRAX/mUSD
Uniswap V3 FRAX/DAI
Sushi FRAX/SUSHI
StakeDAO sdFRAX3CRV-f
Gelato Uniswap FRAX/DAI
StakeDAO sdETH-FraxPut
Uniswap V3 FRAX/agEUR
Vesper Orbit FRAX
Temple FRAX/TEMPLE
Curve VSTFRAX-f


In [125]:
# Get all pools listed in the "Staking" tab
req = cache.get('https://api.frax.finance/pools')
pools = json.loads(req['value'])
'''
# A pool looks like
{'apy': 18.786202278137203,
  'apy_max': 56.35860683441161,
  'chain': 'arbitrum',
  'identifier': 'Curve VSTFRAX-f',
  'is_deprecated': False,
  'liquidity_locked': 24336315.042461667,
  'logo': '',
  'pair': 'Curve VSTFRAX-f',
  'pairLink': 'https://app.frax.finance/staking#Curve_VSTFRAX_F',
  'platform': 'curve_arbi_vstfrax',
  'pool_rewards': ['FXS', 'VSTA'],
  'pool_tokens': ['FRAX', 'VST']},
'''

# Huge PITA but we can get contract addresses for all these pools as well...
req = cache.get('https://raw.githubusercontent.com/FraxFinance/frax-solidity/master/src/types/constants.ts')
constants = req['value'].decode('utf8')
start = constants.find('export const CONTRACT_ADDRESSES')
end = constants.find('export const INVESTOR_ALLOCATIONS')
constants = constants[start:end].replace('export const CONTRACT_ADDRESSES = ', '')

# best tool: https://regex101.com/
# also useful: https://pythex.org/
# docs: https://note.nkmk.me/en/python-str-replace-translate-re-sub/
# https://jsonlint.com/
import re
constants = re.sub(r'//.*', '', constants)
constants = re.sub(r'[\s]*NOTE.*', '', constants)
constants = re.sub(r'([a-zA-Z0-9_]+)[\s]*:\B', r'"\1":', constants)
import ast
contracts = ast.literal_eval(constants)

staking_contracts = {}
for key in contracts:
  if 'staking_contracts' in contracts[key]:
    for contract in contracts[key]['staking_contracts']:
      staking_contracts[contract] = {'chain': key, 'address': contracts[key]['staking_contracts'][contract]}
    
for gauge in gauges:
  try:
    contract = staking_contracts[gauge['name']]
    print(f"{contract['chain']:10} : {gauge['name']:24} : {contract['address']}")
  except:
    print(f'NO CONTRACT for {gauge["name"]}')
    pass

ethereum   : Uniswap V3 FRAX/USDC     : 0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0
polygon    : mStable FRAX/mUSD        : 0xc425Fd9Ed3C892d849C9E1a971516da1C1B29696
ethereum   : Uniswap V3 FRAX/DAI      : 0xF22471AC2156B489CC4a59092c56713F813ff53e
ethereum   : Sushi FRAX/SUSHI         : 0xb4Ab0dE6581FBD3A02cF8f9f265138691c3A7d5D
ethereum   : StakeDAO sdFRAX3CRV-f    : 0xEB81b86248d3C2b618CcB071ADB122109DA96Da2
ethereum   : Gelato Uniswap FRAX/DAI  : 0xcdfc491804A420b677f8e788B5157856910E2F6f
ethereum   : StakeDAO sdETH-FraxPut   : 0x0A53544b2194Dd8Ebc62c779043fc0624705BB56
ethereum   : Uniswap V3 FRAX/agEUR    : 0xf8caEd1943B15B877D7105B9906a618c154f69E8
ethereum   : Vesper Orbit FRAX        : 0x698137C473bc1F0Ea9b85adE45Caf64ef2DF48d6
ethereum   : Temple FRAX/TEMPLE       : 0x10460d02226d6ef7B2419aE150E6377BdbB7Ef16
arbitrum   : Curve VSTFRAX-f          : 0x127963A74c07f72D862F2Bdc225226c3251BD117


In [ ]:
# fraximalist.eth (example) 
address = '0x68e912af6176bb1639aca936eb58d71cc2558b66'